In [42]:
# import selenium drivers and Exceptions
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
# import Database schema object from ConfigRedDB
from configRedDB import RedbusDatabaseConn
#import time module to pause between codes
import time

In [46]:
# initialize the driver for my browser
driver = webdriver.Chrome()
# navigate to the redbus website
driver.get("https://www.redbus.in/")

In [47]:
# function to iterate all view button of government buses and click it to view buses
def viewGovBus(gov_buses):
    # iterate all gov bus tab
        for k in gov_buses:
            # click view buses in that tab
            while True:
                 
                #loops the scrolling till the element is found and clicked 
                 webdriver.ActionChains(driver).move_to_element(k).click(k).perform()

                 # checks if the text changed after clicking, if it does stop scrolling
                 if k.text == "HIDE BUSES":
                      break
                 
                 # Scroll down
                 driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                 # Wait for page to load
                 time.sleep(5)
                 
# Method to scroll down the whole page to load all elements
def scrollToLast(last_height, driver):
    while True:

        # Scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait for page to load
        time.sleep(2)
        # Calculate new scroll height 
        new_height = driver.execute_script("return document.body.scrollHeight")

        # if the new height of the page after loading is equal to last height, it's page and, so stop looping
        if new_height == last_height:
            break
        # update the last height
        last_height = new_height

In [43]:
# initialize the Database connection
cur = RedbusDatabaseConn()

In [48]:
for c in range(2,12):
    card = driver.find_element(By.XPATH, f"//div[@id = 'Carousel']/div[{c}]")
    webdriver.ActionChains(driver).move_to_element(card).click(card).perform()

    # get all the tabs in gov bus page
    tabs = driver.find_elements(By.CSS_SELECTOR, '.DC_117_paginationTable .DC_117_pageTabs')
    #store every tab element text to find it later in every iteration
    tabValues = [int(n.text) for n in tabs]
    # iterate first two tabs in the page
    for t in tabValues:
        # get the tab elemnt by the child number
        tab = driver.find_element(By.XPATH,f'//div[@class="DC_117_paginationTable"]/div[{t}]')

        #clicks a tab
        webdriver.ActionChains(driver).move_to_element(tab).click(tab).perform()

        # Wait for page to load
        time.sleep(3)

        # gets all the routes available 
        titles = driver.find_elements(By.XPATH, '//div[@class="route_link"]/div[@class="route_details"]/a')
        titleLinks = [k.get_attribute('title') for k in titles]

        # iterate first 2 route elements
        for j in titleLinks:
            
            time.sleep(2)

            #  get the tab elemnt again as the DOM is destroyed when page reloads every iteration
            tabnew = driver.find_element(By.XPATH, f'//div[@class="DC_117_paginationTable"]/div[{t}]')
            webdriver.ActionChains(driver).move_to_element(tabnew).click(tabnew).perform()

            # store the route name
            route_name = j
            
            title = driver.find_element(By.XPATH,f"//div[@class='route_details']/a[@title='{j}']")
            route_link = title.get_attribute('href')
            # clicks the route element
            webdriver.ActionChains(driver).move_to_element(title).click(title).perform()

            # Wait for page to load
            time.sleep(5)
            
            # gets all gov bus's view buses button
            gov_buses = driver.find_elements(By.XPATH, '//div[contains(text(), "View Buses")]')

            # function to iterate all button and click it to view buses
            viewGovBus(gov_buses)

            # claculate the initial height before scrolling
            last_height = driver.execute_script("return document.body.scrollHeight")

            # function to scroll to the end of page to load all data including private buses
            scrollToLast(last_height, driver)

            # gets every row of the buses available (state government and private)
            bus_details = driver.find_elements(By.CSS_SELECTOR,".clearfix .row-one")
            time.sleep(10)

            # get the required data from every row in the bus_details
            for i in bus_details:
                
                busname = i.find_element(By.CSS_SELECTOR, ".travels").text
                bustype = i.find_element(By.CSS_SELECTOR, ".bus-type").text
                departing_time = i.find_element(By.CSS_SELECTOR, ".dp-time").text
                duration = i.find_element(By.CSS_SELECTOR, ".dur").text
                reaching_time = i.find_element(By.CSS_SELECTOR, ".bp-time").text

                # assigning zero to buses that has no rating element
                try:
                    result = i.find_element(By.XPATH, ".//div[@class = 'rating-sec lh-24']/div/span | .//div[@class = 'rating-sec']/span[2]").text
                except NoSuchElementException:
                    result = 0.0
                
                # assigning zero to buses that has new rating
                star_rating = 0.0 if result=='New' else float(result)
                price = float(i.find_element(By.XPATH, ".//div[@class = 'seat-fare ']/div[@class='fare d-block']/span").text)
                seats_available = i.find_element(By.CSS_SELECTOR, ".seat-left").text.split()[0]
                # store all the data in the SQL redbusDB database
                cur.Add(route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available)
                

            driver.back()
            driver.back()
            time.sleep(2)
    
    driver.back()
    time.sleep(3)

In [49]:
res = cur.ListRequest()
r = res.fetchall()
for i in range(len(r)):
    print(tuple(r[i]))




(2238, 'Bangalore to Kozhikode', 'https://www.redbus.in/bus-tickets/bangalore-to-kozhikode', 'KSRTC (Kerala) - 581', 'Super Deluxe Non AC Seater Air Bus (2+2)', '10:03', '08h 45m', '18:48', 4.1, 469, 2)
(2239, 'Bangalore to Kozhikode', 'https://www.redbus.in/bus-tickets/bangalore-to-kozhikode', 'KSRTC (Kerala) - 52', 'SWIFT-GARUDA A/C SEATER BUS', '12:01', '07h 54m', '19:55', 3.3, 627, 29)
(2240, 'Bangalore to Kozhikode', 'https://www.redbus.in/bus-tickets/bangalore-to-kozhikode', 'KSRTC (Kerala) - 1378', 'AC MULTI AXLE', '13:46', '07h 04m', '20:50', 2.7, 729, 24)
(2241, 'Bangalore to Kozhikode', 'https://www.redbus.in/bus-tickets/bangalore-to-kozhikode', 'KSRTC (Kerala) - 129', 'SWIFT-GARUDA A/C SEATER BUS', '14:15', '06h 44m', '20:59', 4.0, 627, 32)
(2242, 'Bangalore to Kozhikode', 'https://www.redbus.in/bus-tickets/bangalore-to-kozhikode', 'KSRTC (Kerala) - 2472', 'Garuda Premium', '14:30', '07h 30m', '22:00', 3.7, 1212, 22)
(2243, 'Bangalore to Kozhikode', 'https://www.redbus.in/bu

In [50]:
cur.close()

In [44]:
# cur.deleteAll()